In [3]:
import sys
sys.path.append("../src")
import pandas as pd
import Preprocessing

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.compose import make_column_transformer
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer

/home/bigdata/anaconda3/lib/python3.8/site-packages/spacy/util.py:275: UserWarning: [W031] Model 'en_core_web_sm' (2.2.5) requires spaCy v2.2 and is incompatible with the current spaCy version (2.3.5). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [ ]:
import pandas as pd
import sys
from Features import buildFeatures
from Pipeline import BuildPipeline
from Preprocessing import Preprocessing
from sklearn.svm import LinearSVC, LinearSVR
from xgboost import XGBRegressor, XGBClassifier
from sklearn.linear_model import SGDClassifier, SGDRegressor

class RequestMapper:
    
    def __init__(self, pipelines):
        # storing and unpacking the pipelines
        self.pipelines = pipelines
        self.pipeline_mapping = {\"age\": pipelines.values[0],modelling.predict_text(tfidf_transformed_text)[0]\n    \n    def predict_numerical(self, target_variable, text, age=None, sign=None, gender=None, topic=None):\n        # Build dataframe for transformer\n        columns = self.pipeline_mapping[target_variable].transformation.X_test_numerical.columns\n        entry = pd.DataFrame(columns=columns)\n        \n        data_dict = buildFeatures(text)\n        for name, variable in zip([\"age\",\"sign\",\"gender\",\"topic\"],[age, sign, gender, topic]):\n            if variable != None:\n                if not name == target_variable:\n                    data_dict[name] = variable\n                else:\n                    print(\"Error, target variable is not empty\")\n                \n            elif name != target_variable:\n                print(\"Error, feature missing\")\n        \n        entry_numerical  = entry.append(data_dict, ignore_index=True)\n        \n        data_transformed =  self.pipeline_mapping[target_variable].\\\n                                transformation.numerical_transformer.transform(entry_numerical)\n        \n        return self.pipeline_mapping[target_variable].modelling.predict_numerical(data_transformed)[0]\n            \n        \n    def predict_weighted(self, target_variable, text, age=None, sign=None, gender=None, topic=None):\n        # Build dataframe for transformer\n        data_dict = buildFeatures(text)\n        data_dict[\"text_preprocessed\"] = self.preprocessor.ProcessOne(text)\n        for name, variable in zip([\"age\",\"sign\",\"gender\",\"topic\"],[age, sign, gender, topic]):\n            if variable != None:\n                if not name == target_variable:\n                    data_dict[name] = variable\n                else:\n                    print(\"Error, target variable is not empty\")\n                \n            elif name != target_variable:\n                print(\"Error, feature missing\")\n        \n        prediction = self.pipeline_mapping[target_variable].modelling.weighted_prediction(data_dict,\\\n                                                                                          algo_type=self.pipeline_mapping[target_variable]\\\n                                                                                          .algo_type)\n        return prediction[0]", "format": "text", "mimetype": "text/x-python", "size": 3511, "writable": true, "type": "file"}                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

In [4]:
# read the dataset
df_full_preprocessed = pd.read_pickle("./df_full_preprocessed.pkl")
# import preprocessor
preprocessing = Preprocessing.Preprocessing()

# clean / filter data
df_full_preprocessed = df_full_preprocessed[df_full_preprocessed["text_preprocessed"].str.count(r"[a-zA-Z]") >= 50]
df_full_preprocessed = df_full_preprocessed.drop_duplicates("text_preprocessed")
df_full_preprocessed = df_full_preprocessed[df_full_preprocessed["text_preprocessed"].apply(lambda x: len(x.split())) >= 2]

In [5]:
# use the  text transformer class to create two transformers for the textual and the numerical model
text_transformer = TfidfVectorizer(ngram_range=(1,1), min_df=int(len(df_full_preprocessed)**(1/4)), use_idf=True)
numerical_transformer = make_column_transformer((OneHotEncoder(handle_unknown="ignore"), ["gender","topic","sign"])\
                                                           , remainder=StandardScaler())


In [6]:
# create dataset
X_text = text_transformer.fit_transform(df_full_preprocessed["text_preprocessed"])
X_numerical = numerical_transformer.fit_transform(df_full_preprocessed.drop("text_preprocessed", axis=1))

In [7]:
knn_text = NearestNeighbors(n_neighbors=10)
knn_text.fit(X_text)

NearestNeighbors(n_neighbors=10)

In [8]:
knn_numerical = NearestNeighbors(n_neighbors=10)
knn_numerical.fit(X_numerical)

NearestNeighbors(n_neighbors=10)

In [13]:
def get_k_nearest_texts(knn, vectorizer, df, text):
    text = preprocessing.ProcessOne(text)
    prediction = knn.kneighbors(vectorizer.transform([text]))
    return df.iloc[prediction[1][0]]

def get_k_nearest_numerical(knn, vectorizer, text, age, sign, gender, topic):
    feature

In [2]:
%history -g -f "history.txt"

In [10]:
text = """In information retrieval, tf–idf, TF*IDF, or TFIDF, short for term frequency–inverse document frequency, is a numerical statistic that is intended to reflect how important a word is to a document in a collection or corpus.[1] It is often used as a weighting factor in searches of information retrieval, text mining, and user modeling. The tf–idf value increases proportionally to the number of times a word appears in the document and is offset by the number of documents in the corpus that contain the word, which helps to adjust for the fact that some words appear more frequently in general. tf–idf is one of the most popular term-weighting schemes today. A survey conducted in 2015 showed that 83% of text-based recommender systems in digital libraries use tf–idf.[2]

Variations of the tf–idf weighting scheme are often used by search engines as a central tool in scoring and ranking a document's relevance given a user query. tf–idf can be successfully used for stop-words filtering in various subject fields, including text summarization and classification.

One of the simplest ranking functions is computed by summing the tf–idf for each query term; many more sophisticated ranking functions are variants of this simple mode"""

In [11]:
test = get_k_nearest_texts(knn_text, text_transformer, df_full_preprocessed, text)

information retrieval tf idf tfidf short term document frequency a numerical statistic intend reflect important a word a document a collection corpus 1 a weight factor search information retrieval text mining user modeling increase proportionally number time a word appear document offset number document corpus word help adjust fact word frequently general popular term - weight scheme today a survey conduct 2015 show 83 text - base recommender system digital library 2 variation weight scheme search engine a central tool score rank a document 's relevance a user query successfully stop - word filter subject field include text summarization classification simplest rank function compute sum query term sophisticated rank function variant simple mode 


In [12]:
test

,gender,age,topic,sign,Text length,Number URLs,Number mails,Uppercase ratio,Lowercase ratio,Number ratio,...,Average letters per word,Variance of letters per word,Unique words ratio,Average letters per sentence,Variance of letters per sentence,Average words per sentence,Variance of words per sentence,Maximal uppercase ratio per sentence,Length of the maximal uppercase ratio sentence,text_preprocessed
157453,female,14,Student,Capricorn,89,0,0,0.000000,0.651685,0.000000,...,15.000000,67.500000,1.000000,81.0,0.00,4.0,0.00,0.000000,81,iiiiiiii lllloooooovvvvvvvveeeeeeeee uuuuuu aa...
426419,female,33,indUnk,Capricorn,109,2,0,0.045872,0.678899,0.000000,...,7.181818,13.966942,0.909091,101.0,0.00,11.0,0.00,0.049505,101,arneitai kathgorhmatika lhpsh apagorevmenwn ou...
390211,male,13,indUnk,Pisces,90,0,0,0.577778,0.000000,0.000000,...,33.500000,6.250000,1.000000,40.5,1560.25,1.5,0.25,0.650000,80,schooooooooooooooooooooooooooooool 's outttttt...
444164,male,15,Student,Capricorn,426,0,0,0.438967,0.464789,0.016432,...,33.166667,111.472222,1.000000,211.0,44100.00,6.5,30.25,0.444181,421,talklistenproblemsattireattitudebrainstorms tu...
40247,male,15,indUnk,Capricorn,127,0,0,0.118110,0.472441,0.000000,...,21.400000,285.440000,1.000000,58.5,1640.25,2.5,2.25,0.151515,99,lalalalalalalalalalalalalalalalalalalalala lal...
54565,male,23,indUnk,Sagittarius,114,0,0,0.008772,0.719298,0.026316,...,46.500000,552.250000,1.000000,52.0,324.00,1.0,0.00,0.014286,70,elo2allveteranbloggers thanx4invitingmerox.wal...
88023,female,23,indUnk,Capricorn,320,0,0,0.021875,0.253125,0.000000,...,6.382353,9.118512,0.617647,299.0,0.00,34.0,0.00,0.023411,299,wisteria wisteria wisteria wisteria wisteria w...
570141,female,24,RealEstate,Capricorn,106,0,0,0.000000,0.745283,0.000000,...,86.000000,0.000000,1.000000,97.0,0.00,1.0,0.00,0.000000,97,ninehoursofschool twodaysaweek foranentireseme...
334264,male,14,indUnk,Aquarius,619,0,0,0.000000,0.969305,0.000000,...,55.000000,6.727273,1.000000,618.0,0.00,11.0,0.00,0.000000,618,brandonbiebelbrianandersonmccrankerickostonpau...
55870,male,25,Military,Virgo,2737,0,0,0.000000,0.892948,0.000000,...,2726.000000,0.000000,1.000000,2733.0,0.00,1.0,0.00,0.000000,2733,lglhfljfljhgkl jlh ljh lkjlkclglhfljfljhgkl jl...


In [ ]:
test["text_preprocessed"].valaues[-1]